# Base Notebook
Aim to modularize and modulerize the workflow.
All development should be done in this notebook.
When training, fork one, configurate the config session only, run.


In [ ]:
import os
import csv
import random
import pydicom
import numpy as np
import pandas as pd
from skimage import io
from skimage import measure
from skimage.transform import resize

import tensorflow as tf
from tensorflow import keras

from matplotlib import pyplot as plt
import matplotlib.patches as patches

# Config Session
All modules, including data prep, model, training modules, are going to be defined in abstract layer functions.
Main body calls actual function according to indices specified here. 

*_func_idx is an index of function to be used in *_func_list
*_param_idx is an index of paramters, in tuple, to be used in *_func_list[*_func_idx]

In [ ]:
#we may define model names here for easier reference, to be used in next block
RESNET5 = 1
UNET = 2

#Data preparation options
NO_MOD = 1

#loss functions options
BCE_IOU = 1

In [ ]:
#only change this cell after forking out, values are indices. Max values depends on number of functions defined. 
# 0 is example function. Do not use 0 for function in actual run.
(data_func_idx, model_func_idx, loss_func_idx) = (NO_MOD,UNET,BCE_IOU)
(data_param_idx, model_param_idx, loss_param_idx) = (0,0,0)
EPOCHS = 1

# Abstract Layer Definition
Define list of function pointers, and paramters used by each function. Do not change this block below

*_func_list is an 1 D array of function pointers

*_param_list is a 2D array of tuples containing paramters

In [ ]:
data_func_list = []
model_func_list = []
loss_func_list = []
data_param_list = []
model_param_list = []
loss_param_list = []

### Example for adding data preprocessing function
This section shows a dummy example how a data preprocessing function and paramters are added to abstract layer

In [ ]:
#Add data prep function
def data_func0():
    #load model_param
    (param0,param1) = data_param_list[data_func_idx][data_param_idx]
    print('data_func0: param0[%d], param1[%d]'%(param0,param1))

#add data prep function and paramters 
i = len(data_func_list)                #find idx of next entry
data_func_list.append(data_func0)     #append the function pointer to the list, replace with your function name
data_param_list.append([])             #keep this line even if there's no param
data_param_list[i].append((0,0))       #append as many parameter tuples as you want

### Example for adding model function
This section shows a dummy example how a model function and paramters are added to abstract layer


In [ ]:
#Add a new model
def model_func0(input_size):
    #load model_param
    (param0,param1) = model_param_list[model_func_idx][model_param_idx]
    print('model_func0: param0[%d], param1[%d]'%(param0,param1))
    
    #build model - dummy example
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = keras.layers.Conv2D(1, 1, activation='sigmoid')(inputs)
    outputs = keras.layers.UpSampling2D(param0**param1)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    return model

#add model and paramters 
i = len(model_func_list)                #find idx of next entry
model_func_list.append(model_func0)     #append the function pointer to the list, replace with your function name
model_param_list.append([])             #keep this line even if there's no param
model_param_list[i].append((0,0))       #append as many parameter tuples as you want
model_param_list[i].append((0,1))
model_param_list[i].append((0,2))


### Example for adding loss function
This section shows a dummy example how a loss function and paramters are added to abstract layer


In [ ]:
#add a loss function
def loss_func0(y_true, y_pred):
    #load model_param
    (param0,param1) = loss_param_list[loss_func_idx][loss_param_idx]
    print('model_func1: param0[%d], param1[%d]'%(param0,param1))
    
#add loss function and paramters 
i = len(loss_func_list)                #find idx of next entry
loss_func_list.append(loss_func0)      #append the function pointer to the list, replace with your function name
loss_param_list.append([])             #keep this line even if there's no param
loss_param_list[i].append((10,1))   #append as many parameter tuples as you want
loss_param_list[i].append((10,2))
loss_param_list[i].append((10,3))

# Build Data Preprocessing Models

## Load pneumonia locations

Table contains [filename : pneumonia location] pairs per row. 
* If a filename contains multiple pneumonia, the table contains multiple rows with the same filename but different pneumonia locations. 
* If a filename contains no pneumonia it contains a single row with an empty pneumonia location.

The code below loads the table and transforms it into a dictionary. 
* The dictionary uses the filename as key and a list of pneumonia locations in that filename as value. 
* If a filename is not present in the dictionary it means that it contains no pneumonia.

In [ ]:
# empty dictionary
pneumonia_locations = {}
# load table
with open(os.path.join('../input/stage_1_train_labels.csv'), mode='r') as infile:
    # open reader
    reader = csv.reader(infile)
    # skip header
    next(reader, None)
    # loop through rows
    for rows in reader:
        # retrieve information
        filename = rows[0]
        location = rows[1:5]
        pneumonia = rows[5]
        # if row contains pneumonia add label to dictionary
        # which contains a list of pneumonia locations per filename
        if pneumonia == '1':
            # convert string to float to int
            location = [int(float(i)) for i in location]
            # save pneumonia location in dictionary
            if filename in pneumonia_locations:
                pneumonia_locations[filename].append(location)
            else:
                pneumonia_locations[filename] = [location]

## Data Preparation 1

### Data generator

The dataset is too large to fit into memory, so we need to create a generator that loads data on the fly.

* The generator takes in some filenames, batch_size and other parameters.

* The generator outputs a random batch of numpy images and numpy masks.

In [ ]:
class generator(keras.utils.Sequence):
    
    def __init__(self, folder, filenames, pneumonia_locations=None, batch_size=32, image_size=256, shuffle=True, augment=False, predict=False):
        self.folder = folder
        self.filenames = filenames
        self.pneumonia_locations = pneumonia_locations
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.augment = augment
        self.predict = predict
        self.on_epoch_end()
        
    def __load__(self, filename):
        # load dicom file as numpy array
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        # create empty mask
        msk = np.zeros(img.shape)
        # get filename without extension
        filename = filename.split('.')[0]
        # if image contains pneumonia
        if filename in self.pneumonia_locations:
            # loop through pneumonia
            for location in self.pneumonia_locations[filename]:
                # add 1's at the location of the pneumonia
                x, y, w, h = location
                msk[y:y+h, x:x+w] = 1
        # resize both image and mask
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        msk = resize(msk, (self.image_size, self.image_size), mode='reflect') > 0.5
        # if augment then horizontal flip half the time
        if self.augment and random.random() > 0.5:
            img = np.fliplr(img)
            msk = np.fliplr(msk)
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        msk = np.expand_dims(msk, -1)
        return img, msk
    
    def __loadpredict__(self, filename):
        # load dicom file as numpy array
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        # resize image
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        return img
        
    def __getitem__(self, index):
        # select batch
        filenames = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        # predict mode: return images and filenames
        if self.predict:
            # load files
            imgs = [self.__loadpredict__(filename) for filename in filenames]
            # create numpy batch
            imgs = np.array(imgs)
            return imgs, filenames
        # train mode: return images and masks
        else:
            # load files
            items = [self.__load__(filename) for filename in filenames]
            # unzip images and masks
            imgs, msks = zip(*items)
            # create numpy batch
            imgs = np.array(imgs)
            msks = np.array(msks)
            return imgs, msks
        
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.filenames)
        
    def __len__(self):
        if self.predict:
            # return everything
            return int(np.ceil(len(self.filenames) / self.batch_size))
        else:
            # return full batches only
            return int(len(self.filenames) / self.batch_size)
        
def data_func1():
    #load data param
    n_valid_samples = data_param_list[data_func_idx][data_param_idx]
    print('Number of Validation Samples [%d]'%(n_valid_samples))
    
    # create train and validation generators
    # load and shuffle filenames
    folder = '../input/stage_1_train_images'
    filenames = os.listdir(folder)
    random.shuffle(filenames)
    # split into train and validation filenames
    train_filenames = filenames[n_valid_samples:]
    valid_filenames = filenames[:n_valid_samples]
    print('n train samples', len(train_filenames))
    print('n valid samples', len(valid_filenames))
    n_train_samples = len(filenames) - n_valid_samples
    
    folder = '../input/stage_1_train_images'
    train_gen = generator(folder, train_filenames, pneumonia_locations, batch_size=32, image_size=256, shuffle=True, augment=True, predict=False)
    valid_gen = generator(folder, valid_filenames, pneumonia_locations, batch_size=32, image_size=256, shuffle=False, predict=False)
    return (train_gen, valid_gen)

#add data prep function and paramters 
i = len(data_func_list)                #find idx of next entry
data_func_list.append(data_func1)     #append the function pointer to the list, replace with your function name
data_param_list.append([])             #keep this line even if there's no param
data_param_list[i].append(2560)        #keep this line even if there's no param
data_param_list[i].append(5120)        #keep this line even if there's no param

## Data Preparation 2

In [ ]:
# put code here

#add data prep function and paramters 

# Build Models

## Model 1 - RESNET5 CNN

In [ ]:
def create_downsample(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 1, padding='same', use_bias=False)(x)
    x = keras.layers.MaxPool2D(2)(x)
    return x

def create_resblock(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    return keras.layers.add([x, inputs])

def create_network(input_size, channels, n_blocks=2, depth=4):
    # input
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(inputs)
    # residual blocks
    for d in range(depth):
        channels = channels * 2
        x = create_downsample(channels, x)
        for b in range(n_blocks):
            x = create_resblock(channels, x)
    # output
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(1, 1, activation='sigmoid')(x)
    outputs = keras.layers.UpSampling2D(2**depth)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

def model_function1(input_size):
    #load model_param
    (channels,n_blocks,depth) = model_param_list[model_func_idx][model_param_idx]
    print('model_function1 RESNET-5: channels[%d], n_blocks[%d], depth[%d]'% (channels,n_blocks,depth))
    return create_network(input_size, channels, n_blocks, depth)

#add model and paramters 
i = len(model_func_list)                #find idx of next entry
model_func_list.append(model_function1)     #append the function pointer to the list, replace with your function name
model_param_list.append([])             #keep this line even if there's no param
model_param_list[i].append((32,2,4))       #append as many parameter tuples as you want

## Model 2 - UNET

In [ ]:
#put code here
def build_UNet2D_4L(inp_shape, k_size=3):
    merge_axis = -1 # Feature maps are concatenated along last axis (for tf backend)
    data = keras.Input(shape=inp_shape)
    conv1 = keras.layers.Convolution2D(filters=32, kernel_size=k_size, padding='same', activation='relu')(data)
    conv1 = keras.layers.Convolution2D(filters=32, kernel_size=k_size, padding='same', activation='relu')(conv1)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = keras.layers.Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(pool1)
    conv2 = keras.layers.Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(conv2)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = keras.layers.Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(pool2)
    conv3 = keras.layers.Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(conv3)
    pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = keras.layers.Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(pool3)
    conv4 = keras.layers.Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(conv4)
    pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = keras.layers.Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(pool4)

    up1 = keras.layers.UpSampling2D(size=(2, 2))(conv5)
    conv6 = keras.layers.Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(up1)
    conv6 = keras.layers.Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(conv6)
    merged1 = keras.layers.concatenate([conv4, conv6], axis=merge_axis)
    conv6 = keras.layers.Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(merged1)

    up2 = keras.layers.UpSampling2D(size=(2, 2))(conv6)
    conv7 = keras.layers.Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(up2)
    conv7 = keras.layers.Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(conv7)
    merged2 = keras.layers.concatenate([conv3, conv7], axis=merge_axis)
    conv7 = keras.layers.Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(merged2)

    up3 = keras.layers.UpSampling2D(size=(2, 2))(conv7)
    conv8 = keras.layers.Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(up3)
    conv8 = keras.layers.Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(conv8)
    merged3 = keras.layers.concatenate([conv2, conv8], axis=merge_axis)
    conv8 = keras.layers.Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(merged3)

    up4 = keras.layers.UpSampling2D(size=(2, 2))(conv8)
    conv9 = keras.layers.Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(up4)
    conv9 = keras.layers.Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(conv9)
    merged4 = keras.layers.concatenate([conv1, conv9], axis=merge_axis)
    conv9 = keras.layers.Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(merged4)

    conv10 = keras.layers.Convolution2D(filters=1, kernel_size=k_size, padding='same', activation='sigmoid')(conv9)

    output = conv10
    model = keras.Model(data, output)
    return model

def model_function2(input_size):
    #load model_param
    k_size = model_param_list[model_func_idx][model_param_idx]
    print('model_function2 UNET: ksize[%d]'% (k_size))
    
    inp_shape = (input_size,input_size, 1)
    return build_UNet2D_4L(inp_shape, k_size)

#add model and paramters 
i = len(model_func_list)                #find idx of next entry
model_func_list.append(model_function2)     #append the function pointer to the list, replace with your function name
model_param_list.append([])             #keep this line even if there's no param
model_param_list[i].append((3))       #append as many parameter tuples as you want
#add model and paramters 

## Model 3- TBD

In [ ]:
#put code here

#add model and paramters 

# Build Loss Function


## Loss function 1 - IOU + BCE

In [ ]:
# define iou or jaccard loss function
def iou_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true * y_pred)
    score = (intersection + 1.) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection + 1.)
    return 1 - score

# combine bce loss and iou loss
def iou_bce_loss(y_true, y_pred):
     #load model_param
    (bce_weight,iou_weight) = loss_param_list[loss_func_idx][loss_param_idx]
    print('iou_bce_loss: bce_weight[%f], iou_weight[%f]'% (bce_weight,iou_weight))
    return bce_weight * keras.losses.binary_crossentropy(y_true, y_pred) + iou_weight * iou_loss(y_true, y_pred)

#add loss function and paramters 
i = len(loss_func_list)                #find idx of next entry
loss_func_list.append(iou_bce_loss)      #append the function pointer to the list, replace with your function name
loss_param_list.append([])             #keep this line even if there's no param
loss_param_list[i].append((0.5,0.5))   #append as many parameter tuples as you want
loss_param_list[i].append((0.6,0.4))
loss_param_list[i].append((0.4,0.6))

# Complete Abstract Layer
Assign selected function pointers to common function pointer to be used in following code in main body

In [ ]:
#Assign selected function pointer to main function pointer
data_func = data_func_list[data_func_idx]
model_func = model_func_list[model_func_idx]
loss_func = loss_func_list[loss_func_idx]

# Prepare Additional Static Training Factors
For the ones not requiring tuning

In [ ]:
# mean iou as a metric
def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersect = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3])
    smooth = tf.ones(tf.shape(intersect))
    return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))

# cosine learning rate annealing
def cosine_annealing(x):
    lr = 0.001
    epochs = EPOCHS
    return lr*(np.cos(np.pi*x/epochs)+1.)/2
learning_rate = tf.keras.callbacks.LearningRateScheduler(cosine_annealing)

# Training
Use abstracted functions in training

In [ ]:
# create network and compiler
model = model_func(input_size=256)
model.compile(optimizer='adam',
              loss=loss_func,
              metrics=['accuracy', mean_iou])

# create data set
(train_gen, valid_gen) = data_func()

## Start Training

In [ ]:
history = model.fit_generator(train_gen, validation_data=valid_gen, callbacks=[learning_rate], epochs=EPOCHS, workers=4, use_multiprocessing=True)

## Review History

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(131)
plt.plot(history.epoch, history.history["loss"], label="Train loss")
plt.plot(history.epoch, history.history["val_loss"], label="Valid loss")
plt.legend()
plt.subplot(132)
plt.plot(history.epoch, history.history["acc"], label="Train accuracy")
plt.plot(history.epoch, history.history["val_acc"], label="Valid accuracy")
plt.legend()
plt.subplot(133)
plt.plot(history.epoch, history.history["mean_iou"], label="Train iou")
plt.plot(history.epoch, history.history["val_mean_iou"], label="Valid iou")
plt.legend()
plt.show()

## Validate results

In [ ]:
for imgs, msks in valid_gen:
    # predict batch of images
    preds = model.predict(imgs)
    # create figure
    f, axarr = plt.subplots(4, 8, figsize=(20,15))
    axarr = axarr.ravel()
    axidx = 0
    # loop through batch
    for img, msk, pred in zip(imgs, msks, preds):
        # plot image
        axarr[axidx].imshow(img[:, :, 0])
        # threshold true mask
        comp = msk[:, :, 0] > 0.5
        # apply connected components
        comp = measure.label(comp)
        # apply bounding boxes
        predictionString = ''
        for region in measure.regionprops(comp):
            # retrieve x, y, height and width
            y, x, y2, x2 = region.bbox
            height = y2 - y
            width = x2 - x
            axarr[axidx].add_patch(patches.Rectangle((x,y),width,height,linewidth=2,edgecolor='b',facecolor='none'))
        # threshold predicted mask
        comp = pred[:, :, 0] > 0.5
        # apply connected components
        comp = measure.label(comp)
        # apply bounding boxes
        predictionString = ''
        for region in measure.regionprops(comp):
            # retrieve x, y, height and width
            y, x, y2, x2 = region.bbox
            height = y2 - y
            width = x2 - x
            axarr[axidx].add_patch(patches.Rectangle((x,y),width,height,linewidth=2,edgecolor='r',facecolor='none'))
        axidx += 1
    plt.show()
    # only plot one batch
    break

# Predict test images

In [ ]:
# load and shuffle filenames
folder = '../input/stage_1_test_images'
test_filenames = os.listdir(folder)
print('n test samples:', len(test_filenames))

# create test generator with predict flag set to True
test_gen = generator(folder, test_filenames, None, batch_size=25, image_size=256, shuffle=False, predict=True)

# create submission dictionary
submission_dict = {}
# loop through testset
for imgs, filenames in test_gen:
    # predict batch of images
    preds = model.predict(imgs)
    # loop through batch
    for pred, filename in zip(preds, filenames):
        # resize predicted mask
        pred = resize(pred, (1024, 1024), mode='reflect')
        # threshold predicted mask
        comp = pred[:, :, 0] > 0.5
        # apply connected components
        comp = measure.label(comp)
        # apply bounding boxes
        predictionString = ''
        for region in measure.regionprops(comp):
            # retrieve x, y, height and width
            y, x, y2, x2 = region.bbox
            height = y2 - y
            width = x2 - x
            # proxy for confidence score
            conf = np.mean(pred[y:y+height, x:x+width])
            # add to predictionString
            predictionString += str(conf) + ' ' + str(x) + ' ' + str(y) + ' ' + str(width) + ' ' + str(height) + ' '
        # add filename and predictionString to dictionary
        filename = filename.split('.')[0]
        submission_dict[filename] = predictionString
    # stop if we've got them all
    if len(submission_dict) >= len(test_filenames):
        break

# save dictionary as csv file
sub = pd.DataFrame.from_dict(submission_dict,orient='index')
sub.index.names = ['patientId']
sub.columns = ['PredictionString']
sub.to_csv('submission.csv')